<a href="https://colab.research.google.com/github/rameshhpathak/DisasterAssessment/blob/master/segmentation_major.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



    100% |████████████████████████████████| 993kB 20.4MB/s 


In [0]:
Basic_blocks = drive.CreateFile({'id':'1LIJahCBOgKswPjV3dCdjno6iQunL1JQN'})
FusionNet = drive.CreateFile({'id':'1tEd6iRLRH-aQO7dzZkkYnZ4sYDVSaNNz'})
UNet = drive.CreateFile({'id':'1Xf0xSo-CswXiuiUR_tc9IbvzRx-UWU7M'})

In [0]:
Basic_blocks.GetContentFile('Basic_blocks.py')
FusionNet.GetContentFile('FusionNet.py')
UNet.GetContentFile('UNet.py')

In [0]:
from FusionNet import *
from UNet import *
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# hyperparameters

batch_size = 5
img_size = 256
lr = 0.001
epoch = 50

In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# input pipeline

img_dir = "/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/Roads"

img_data = dset.ImageFolder(root=img_dir, transform=transforms.Compose([
    transforms.Resize(size=img_size),
    transforms.CenterCrop(size=(img_size, img_size * 2)),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=5)

In [10]:
torch.cuda.get_device_name(0)


'Tesla K80'

In [11]:
generator = FusionGenerator(3, 3, 64).cuda()


------Initiating FusionNet------



In [12]:
try:
  state = torch.load('/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/segmentation/model/fusionnet.pth')
  print('\n-------model restored--------\n')
  generator.load_state_dict(state['state_dict'])
  gen_optimizer.load_state_dict(state['optimizer'])
  epoch_now = state['epoch']
  
except:
  print('\n-------model not restored----\n')
  gen_optimizer = torch.optim.Adam(generator.parameters(), lr= lr)
  epoch_now = 0
  pass


-------model not restored----



In [0]:
recon_loss_func = nn.MSELoss()

In [0]:
loc = '/content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/segmentation/model/'

In [0]:
file = open(loc + 'fusionnet_mse_loss2', 'w')

In [16]:

len(img_batch)

798

In [0]:
import matplotlib.pyplot as plt

In [0]:
loss_in_epoch = []
for i in range(epoch_now, epoch):
  total_loss = 0
  for _, (image, label) in enumerate(img_batch):
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)

        gen_optimizer.zero_grad()

        x = Variable(satel_image).cuda(0)
        y_ = Variable(map_image).cuda(0)
        y = generator.forward(x)

        loss = recon_loss_func(y, y_)
        file.write(str(loss) + "\n")
        loss.backward()
        gen_optimizer.step()
        
        total_loss += loss.item()

        if _ % 800 == 0:
            print(i)
            print(loss)
            v_utils.save_image(x.cpu().data, loc + "result/original_image_{}_{}.png".format(i, _))
            v_utils.save_image(y_.cpu().data, loc + "result/label_image_{}_{}.png".format(i, _))
            v_utils.save_image(y.cpu().data, loc + "result/gen_image_{}_{}.png".format(i, _))
            
          
  state = {
      'epoch': i+1,
      'state-dict': generator.state_dict(),
      'optimizer': gen_optimizer.state_dict(),
      'losslogger': losslogger,
       }
  
  torch.save(state, loc + 'fusionnet.pth')
  
  loss_in_epoch.append(total_loss/len(img_batch))
  
  if i % 10 == 0:
    plt.plot(loss_in_epoch)
    plt.savefig(loc + 'loss/loss_in_{}'.format(i))
  

   
           



0
tensor(0.1000, device='cuda:0', grad_fn=<MseLossBackward>)
